In [108]:
import requests
from datetime import date, timedelta
import pprint
import pandas as pd
from time import sleep
import yfinance as yf
!pip install pandas_ta
import pandas_ta as ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=0d252fc1871dc432f0ed2c49ed737c247cc55370b71f8330c09cb4db794ba23e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


**Download News Sentiment Meethod**

In [3]:
def pull_news(cur_date,ticker,limit=100):
  api_key = "DJ6ADDN7HUrgAXHVwxQ0vBUrtpIykzXy"
  news = []
  url = f"https://api.polygon.io/v2/reference/news?ticker={ticker}&limit=100&published_utc={cur_date}&apiKey={api_key}"
  response = requests.get(url)


  if response.status_code == 200:
    data = response.json()
    # pprint.pprint(data)
    results = data.get("results", [])
    # pprint.pprint(results)

    for article in results:
      news.append(article)


    else: print(response.status_code)
    # sleep(60)

  return news


In [4]:
from datetime import datetime, timedelta

start_date = datetime.strptime("2024-04-03", "%Y-%m-%d")
end_date = datetime.strptime("2024-04-04", "%Y-%m-%d")

date_range = []
current_date = start_date

while current_date <= end_date:
    date_range.append(current_date.strftime("%Y-%m-%d"))
    current_date += timedelta(days=1)


In [ ]:
from time import sleep
aapl_news = []
spy_news  = []
for date in date_range:
  aapl_news.append(pull_news(date,"AAPL"))
  spy_news.append(pull_news(date,"SPY"))
  # sleep(60/5*2)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [ ]:
sentiment = {}

for day in aapl_news:
    for article in day:
        cur_date = article.get("published_utc")[:10]
        cur_description = article.get("description")
        value = nlp(cur_description)
        print(value)
        sentiment.setdefault(cur_date, {"neg": 0, "pos": 0})
        val = value[0].get("label")
        score = float(value[0].get("score"))
        print(score)
        if val == "negative":
            sentiment[cur_date]["neg"] += score
        elif val == "positive":
            sentiment[cur_date]["pos"] += score


In [ ]:
print(sentiment)
aapl = yf.download("AAPL",start=start_date,end=end_date+timedelta(1),progress=False).reset_index()
aapl["pos"], aapl["neg"] = [0]*len(aapl), [0]*len(aapl)
print(aapl)

In [ ]:
for i in range(len(aapl)):
  cur_date = str(datetime.strftime(aapl["Date"][i],"%Y-%m-%d"))
  if cur_date in sentiment:
    aapl["pos"][i] = sentiment[cur_date]["pos"]
    aapl["neg"][i] = sentiment[cur_date]["neg"]


In [ ]:
print(aapl)

In [147]:

aapl = pd.read_csv("/content/raw_news_2.csv")
aapl = aapl[(aapl["Ticker"] == "AAPL") & (aapl["Positive"] + aapl["Negative"] > 0)].reset_index(drop=True)
aapl["ROC"] = aapl["Close"].shift(-1)/aapl["Close"]-1
# Assuming you've already loaded your data into the 'aapl' DataFrame
# Print the modified DataFrame
print(aapl)
aapl = aapl.dropna()
# Print the filtered DataFrame
print(aapl)



           Date Ticker        Open        High         Low       Close  \
0    12/30/2021   AAPL  179.470001  180.570007  178.089996  178.199997   
1    12/31/2021   AAPL  178.089996  179.229996  177.259995  177.570007   
2      1/3/2022   AAPL  177.830002  182.880005  177.710007  182.009995   
3      1/4/2022   AAPL  182.630005  182.940002  179.119995  179.699997   
4      1/5/2022   AAPL  179.610001  180.169998  174.639999  174.919998   
..          ...    ...         ...         ...         ...         ...   
303   5/23/2023   AAPL  173.130005  173.380005  171.279999  171.559998   
304   5/24/2023   AAPL  171.089996  172.419998  170.520004  171.839996   
305   5/25/2023   AAPL  172.410004  173.899994  171.690002  172.990005   
306   5/30/2023   AAPL  176.960007  178.990005  176.570007  177.300003   
307   5/31/2023   AAPL  177.330002  179.350006  176.759995  177.250000   

      Adj Close     Volume      Sector  Positive  Negative   Neutral       ROC  
0    176.186935   59773000  Te

In [148]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you've already loaded your data into 'aapl'
X = aapl[["Positive", "Negative"]]
Y = aapl["ROC"]

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
model = xgb.XGBRegressor(objective="reg:squarederror")  # For regression tasks

# Fit the model to the training data
model.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error: {mse:.4f}")


Mean Squared Error: 0.0009


In [149]:
print(Y_pred)
print(Y)

[-0.0146271   0.01788503  0.00260168 -0.01263983  0.00138683 -0.00389833
  0.01942861 -0.01871114  0.00340011 -0.00491163 -0.00823705 -0.00141775
 -0.00951122  0.00014855  0.00029288 -0.0090176   0.01799224 -0.0146271
 -0.00916618 -0.0325922   0.00119991  0.001716   -0.00389833 -0.00646324
  0.01412083  0.02257626  0.04325802 -0.00227291 -0.01988783 -0.00706911
  0.00092463 -0.03318041  0.01387393  0.0204523  -0.02116739 -0.03085133
 -0.00389833  0.03694133  0.00097057 -0.00314714  0.0244362  -0.00491163
  0.01585262 -0.02356122  0.01093447  0.0037513  -0.00389833 -0.01681654
 -0.00501787 -0.00344847 -0.00501787 -0.00577591 -0.04082998 -0.02475606
 -0.00048616 -0.039113   -0.00389833 -0.01123446  0.03212057 -0.00229407
  0.01018432  0.01826942]
0     -0.003535
1      0.025004
2     -0.012692
3     -0.026600
4     -0.016693
         ...   
302   -0.015155
303    0.001632
304    0.006692
305    0.024915
306   -0.000282
Name: ROC, Length: 307, dtype: float64


In [150]:
aapl['ROC'] = aapl['ROC'].apply(lambda x: 0 if x < 0 else 1)

In [152]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you've already loaded your data into 'aapl'
X = aapl[["Positive", "Negative"]]
Y = aapl["ROC"]  # Assuming ROC is already modified to be binary (0 or 1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
model = xgb.XGBClassifier(objective="binary:logistic")  # For binary classification

# Fit the model to the training data
model.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.4677


In [153]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Normalize features (sentiment scores)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Build the neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Accuracy on test data: {accuracy:.4f}")


Epoch 1/100
8/8 [==============================] - 1s 32ms/step - loss: 0.6997 - accuracy: 0.4531 - val_loss: 0.6953 - val_accuracy: 0.5161
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6914 - accuracy: 0.5143 - val_loss: 0.7087 - val_accuracy: 0.4194
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6895 - accuracy: 0.5184 - val_loss: 0.7230 - val_accuracy: 0.4194
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 0.6782 - accuracy: 0.5592 - val_loss: 0.7315 - val_accuracy: 0.4194
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6776 - accuracy: 0.5510 - val_loss: 0.7386 - val_accuracy: 0.4194
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6768 - accuracy: 0.5510 - val_loss: 0.7441 - val_accuracy: 0.4194
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6818 - accuracy: 0.5469 - val_loss: 0.7479 - val_accuracy: 0.4194
Epoch 8/100
8/8 [========

In [154]:
aapl["MA"] = ta.ma("ema",aapl.Close,length=10)
aapl = aapl.dropna()

In [155]:

# Split data into training and testing sets
X = aapl[["Positive", "Negative","MA"]]
Y = aapl["ROC"]

X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.4, shuffle=False)

# Build the neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Accuracy on test data: {accuracy:.4f}")


Epoch 1/100
6/6 [==============================] - 1s 44ms/step - loss: 0.7055 - accuracy: 0.4551 - val_loss: 0.6940 - val_accuracy: 0.5083
Epoch 2/100
6/6 [==============================] - 0s 10ms/step - loss: 0.6847 - accuracy: 0.5618 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 3/100
6/6 [==============================] - 0s 12ms/step - loss: 0.6804 - accuracy: 0.5562 - val_loss: 0.6941 - val_accuracy: 0.5083
Epoch 4/100
6/6 [==============================] - 0s 10ms/step - loss: 0.6793 - accuracy: 0.5955 - val_loss: 0.6953 - val_accuracy: 0.5000
Epoch 5/100
6/6 [==============================] - 0s 9ms/step - loss: 0.6769 - accuracy: 0.6067 - val_loss: 0.6980 - val_accuracy: 0.4917
Epoch 6/100
6/6 [==============================] - 0s 12ms/step - loss: 0.6697 - accuracy: 0.6348 - val_loss: 0.7005 - val_accuracy: 0.5000
Epoch 7/100
6/6 [==============================] - 0s 12ms/step - loss: 0.6651 - accuracy: 0.6292 - val_loss: 0.7032 - val_accuracy: 0.4917
Epoch 8/100
6/6 [====